In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from RRNN import RRNN
from NeuroTools.signals.spikes import SpikeTrain
import numpy as np

CSAConnector: libneurosim support not available in NEST.
Falling back on PyNN's default CSAConnector.
Please re-compile NEST using --with-libneurosim=PATH
/usr/lib/python2.7/dist-packages/matplotlib/__init__.py:1314: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

/usr/local/lib/python2.7/dist-packages/NeuroTools/__init__.py:125: DependencyWarning: ** interval ** package is not installed.
To have functions using interval please install the package.
website : http://pypi.python.org/pypi/interval/1.0.0



In [3]:
net = RRNN(recurrent=True, ring=False)
df, spikesE, spikesI = net.model()

for spiketrain in spikesE.spiketrains:
    print 'Fano factor : {0}'.format(SpikeTrain(spiketrain).fano_factor_isi())

/home/daav/opt/nest/lib/python2.7/site-packages/nest/lib/hl_api_helper.py:61: UserWarning: 
ConvergentConnect is deprecated and will be removed in a future version of NEST.
Please use Connect instead!
For details, see http://www.nest-simulator.org/connection_management                       
Fano factor : 0.0
Fano factor : 0.0


Exception: No spikes in the SpikeTrain !

Fano factor forces us to manage case of silent neurons. Another possible approach is to compute an equivalent index using cv of collapsed activity of population neurons over cvs mean.

In [ ]:
A = np.array([[1, 2, 6], [1, 3, 6]])

In [ ]:
print (A.flatten())

In [ ]:
print (np.unique(A))

np.unique() is apparently the function to use with our spiketrains

In [ ]:
tab = np.array(spikesE.spiketrains)

In [ ]:
print(tab) 

In [ ]:
tab = np.array([])
for st in spikesE.spiketrains:
    tab = np.append(tab, np.array(st), axis=0)
print tab

In [ ]:
tab2 = np.unique(tab)

In [ ]:
print (tab2)

In [ ]:
cv_collapsedspikes = SpikeTrain(tab2).cv_isi()

In [ ]:
print cv_collapsedspikes

In [ ]:
all_CVs = np.array([])
for st in spikesE.spiketrains :
    all_CVs = np.append(all_CVs, SpikeTrain(np.array(st)).cv_isi())
meanCv = np.nanmean(all_CVs)

In [ ]:
print(meanCv)

In [ ]:
fakefano = cv_collapsedspikes/meanCv

In [ ]:
print fakefano

Now a little test with poisson process

In [ ]:
import pyNN.nest as sim

In [ ]:
sim.setup(timestep=0.1)

poisson = sim.Population(100, sim.SpikeSourcePoisson(rate=10))
poisson.record('spikes')

sim.run(1000)
poisson_spikes = poisson.get_data().segments[0]

tab = np.array([])
for st in poisson_spikes.spiketrains:
    tab = np.append(tab, np.array(st), axis=0)

cv_collapsed = SpikeTrain(np.unique(tab)).cv_isi()

all_CVs = np.array([])
for st in poisson_spikes.spiketrains :
    all_CVs = np.append(all_CVs, SpikeTrain(np.array(st)).cv_isi())

meanCv = np.nanmean(all_CVs)

fakefano = cv_collapsed/meanCv

print fakefano

In [ ]:
np.array(poisson_spikes.spiketrains[1])

## pyspike


In [ ]:
def reliability(spike_data, method='purpura'):
    spike_trains = []    
    for st_ in spike_data:
        spike_trains.append(st_)

    if method=='pyspike':
        import pyspike as spk
        spike_trains = [ spk.SpikeTrain(st_, edges=(0, params['block_duration'])) for st_ in spike_trains]   # spike_trains.append(spk.SpikeTrain(st_, edges=(0, params['block_duration'])))

        #from pyspike import spike_sync_profile as profile # , max_tau=1000
        from pyspike import spike_profile as profile
        spk.disable_backend_warning = True
        avrg_profile = profile(spike_trains).avrg()

    elif method=='purpura':
        import elephant
        from neo.core import SpikeTrain
        from quantities import s

        #spike_trains = [ SpikeTrain(st_*s, t_stop=1*s) for st_ in spike_trains] 
        #for st_ in spike_data[:8]: # HACK to remove the last trials which seem weak
        #    spike_trains.append(SpikeTrain(st_*s, t_stop=params['block_duration']*s))
        D = elephant.spike_train_dissimilarity.victor_purpura_dist(spike_trains)
        avrg_profile = D.mean()

    elif method=='van_rossum':
        import elephant
        from neo.core import SpikeTrain
        from quantities import s

        spike_trains = [ SpikeTrain(st_*s, t_stop=params['block_duration']*s) for st_ in spike_trains] 
        #for st_ in spike_data[:8]: # HACK to remove the last trials which seem weak
        #    spike_trains.append(SpikeTrain(st_*s, t_stop=params['block_duration']*s))
        D = elephant.spike_train_dissimilarity.van_rossum_dist(spike_trains, tau=np.array(1.0) * s)
        avrg_profile = D.mean()

    return avrg_profile
            

print ("reliability = " + "{0:.4f}".format(reliability(poisson_spikes.spiketrains)))


In [ ]:
net = RRNN(recurrent=True, ring=False)
df, spikesE, spikesI = net.model()

In [ ]:
print ("reliability = " + "{0:.4f}".format(reliability(spikesE.spiketrains)))